<a href="https://colab.research.google.com/github/dmitriyGrandi/EventMe-AI/blob/main/EventMe_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot --upgrade
!pip install gigachat
!pip install beautifulsoup4
!pip install langdetect


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.3/737.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 46.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=0c6a1eb6bda461fe6939178b2404389d382c74461cf3c2b71370add29506ba34
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
# ==========================================================
# ============ ОБНОВЛЕНИЕ ДЛЯ COLAB v29.0 (MEGA SEARCH) ====
# ==========================================================
# ===== ✨ ДОБАВЛЕНЫ: ТЕАТРЫ, МУЗЕИ, КИНО, СТАДИОНЫ ✨ =====
# ==========================================================

# --- 0. УСТАНОВКА ЗАВИСИМОСТЕЙ ---
# !pip install python-telegram-bot --upgrade
# !pip install gigachat
# !pip install beautifulsoup4
# !pip install nest_asyncio

# --- 1. ИМПОРТЫ И НАСТРОЙКИ СРЕДЫ ---
import nest_asyncio
nest_asyncio.apply()

import asyncio, requests, re, urllib.parse, random
from bs4 import BeautifulSoup
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.constants import ParseMode
from telegram.ext import (
    Application, CommandHandler, MessageHandler, filters,
    ConversationHandler, ContextTypes, CallbackQueryHandler
)
from gigachat import GigaChat
from google.colab import userdata

# --- 2. МОДУЛЬ ПАРСЕРОВ (РАСШИРЕННАЯ БАЗА) ---

def perform_yandex_search(query: str) -> list:
    print(f"🚀 Поиск на Yandex.Maps: '{query}'...")
    venues = []

    # 1. Попытка реального парсинга
    try:
        encoded_query = urllib.parse.quote(f"Москва {query}")
        url = f"https://yandex.ru/maps/213/moscow/search/{encoded_query}/"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept-Language': 'ru-RU,ru;q=0.9'
        }
        response = requests.get(url, headers=headers, timeout=8)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # Ищем сниппеты организаций
            for item in soup.find_all('li', class_='search-snippet-view')[:5]:
                title_tag = item.find('div', class_='search-business-snippet-view__title')
                if not (title_tag and title_tag.find('a')): continue

                title = title_tag.find('a').get_text(strip=True)
                link = "https://yandex.ru" + title_tag.find('a')['href']

                address_tag = item.find('div', class_='search-business-snippet-view__address')
                address = address_tag.get_text(strip=True) if address_tag else 'Москва'

                venues.append({'title': title, 'link': link, 'description': address})
    except Exception as e:
        print(f"⚠️ Ошибка парсинга (это нормально, включаем резерв): {e}")

    # 2. ЗАПАСНОЙ ПЛАН (FALLBACK - MEGA DATABASE)
    # Если парсинг не нашел мест или заблокирован, берем из базы
    if not venues:
        print("🔧 Включаю режим 'Виртуальный гид' (Расширенная база)...")
        q_lower = query.lower()

        # === КУЛЬТУРА И ИСКУССТВО ===
        if any(w in q_lower for w in ["театр", "спектакль", "драм", "опера"]):
            venues = [
                {'title': 'Большой театр', 'link': 'https://bolshoi.ru', 'description': 'Театральная пл., 1'},
                {'title': 'МХТ им. А.П. Чехова', 'link': 'https://mxat.ru', 'description': 'Камергерский пер., 3'},
                {'title': 'Театр Наций', 'link': 'https://theatreofnations.ru', 'description': 'Петровский пер., 3'},
                {'title': 'Ленком', 'link': 'https://lenkom.ru', 'description': 'Малая Дмитровка, 6'}
            ]
        elif any(w in q_lower for w in ["музей", "выставка", "галерея", "искусств"]):
            venues = [
                {'title': 'Третьяковская галерея', 'link': 'https://tretyakovgallery.ru', 'description': 'Лаврушинский пер., 10'},
                {'title': 'ГМИИ им. Пушкина', 'link': 'https://pushkinmuseum.art', 'description': 'Волхонка, 12'},
                {'title': 'ГЭС-2', 'link': 'https://v-a-c.org/ges2', 'description': 'Болотная наб., 15'},
                {'title': 'Гараж', 'link': 'https://garagemca.org', 'description': 'Парк Горького'}
            ]
        elif any(w in q_lower for w in ["кино", "фильм", "премьера"]):
            venues = [
                {'title': 'Художественный', 'link': 'https://cinema1909.ru', 'description': 'Арбатская пл., 14'},
                {'title': 'КАРО 11 Октябрь', 'link': 'https://karofilm.ru', 'description': 'Новый Арбат, 24'},
                {'title': 'Москва (Кинотеатр)', 'link': 'https://cinema.moscow', 'description': 'Охотный Ряд, 2'}
            ]

        # === СПОРТ ===
        elif any(w in q_lower for w in ["стадион", "матч", "арена", "футбол"]):
            venues = [
                {'title': 'Лужники', 'link': 'https://luzhniki.ru', 'description': 'Лужнецкая наб., 24'},
                {'title': 'ВТБ Арена', 'link': 'https://vtb-arena.com', 'description': 'Ленинградский пр-т, 36'},
                {'title': 'Открытие Банк Арена', 'link': 'https://spartak.com', 'description': 'Волоколамское ш., 69'}
            ]
        elif any(w in q_lower for w in ["спорт", "зал", "фитнес", "качал"]):
            venues = [
                {'title': 'World Class', 'link': 'https://worldclass.ru', 'description': 'Сеть премиум-клубов'},
                {'title': 'DDX Fitness', 'link': 'https://ddxfitness.ru', 'description': 'Доступный фитнес'},
                {'title': 'Crocus Fitness', 'link': 'https://crocusfitness.com', 'description': 'Крокус Сити'},
                {'title': 'Encore Fitness', 'link': 'https://encorefitness.ru', 'description': 'Москва-Сити'}
            ]

        # === ПРИРОДА И ПРОГУЛКИ ===
        elif any(w in q_lower for w in ["парк", "бег", "гулять", "природ", "воздух"]):
            venues = [
                {'title': 'Парк Горького', 'link': 'https://yandex.ru/maps/org/park_gorkogo/1206307458', 'description': 'Ул. Крымский Вал, 9'},
                {'title': 'ВДНХ', 'link': 'https://vdnh.ru', 'description': 'Проспект Мира, 119'},
                {'title': 'Зарядье', 'link': 'https://zaryadyepark.ru', 'description': 'Ул. Варварка, 6'},
                {'title': 'Парк Сокольники', 'link': 'https://parksokolniki.ru', 'description': 'Сокольнический Вал, 1'}
            ]

        # === ЕДА ===
        elif any(w in q_lower for w in ["еда", "ресторан", "кафе", "ужин", "обед", "завтрак"]):
            venues = [
                {'title': 'Депо.Москва', 'link': 'https://depomoscow.ru', 'description': 'Лесная ул., 20'},
                {'title': 'Dr. Живаго', 'link': 'https://drzhivago.ru', 'description': 'Моховая, 15/1'},
                {'title': 'Erwin.Река', 'link': 'https://rekamoreokean.ru', 'description': 'Набережная Тараса Шевченко'},
                {'title': 'Кофемания', 'link': 'https://coffeemania.ru', 'description': 'Сеть ресторанов'}
            ]

        # === ЕСЛИ НИЧЕГО НЕ ПОДОШЛО ===
        else:
            venues = [
                {'title': 'Центр Москвы (Красная Площадь)', 'link': 'https://yandex.ru/maps/213/moscow', 'description': 'Главная достопримечательность'},
                {'title': 'Москва-Сити', 'link': 'https://citymoscow.ru', 'description': 'Пресненская наб.'}
            ]

    # Перемешиваем, чтобы не выдавать одно и то же каждый раз, если берем из базы
    if len(venues) > 1:
        random.shuffle(venues)

    return venues[:5] # Возвращаем топ-5

# --- 3. ТЕКСТЫ И ПРОМПТЫ ---

STRINGS = {
    'ru': {
        'select_lang': 'Выберите язык / Please select a language:',
        'select_budget': "💰 Какой у тебя примерный бюджет на человека?",
        'select_people': "👥 Отлично! А какая у вас компания?",
        'select_duration': "⏳ Понял. На какой срок планируем?",
        'select_interests': "🎯 Супер! Остался последний шаг. \n\nНапиши, чего хочется? (Например: 'хочу в театр', 'найти спортзал', 'сходить в музей' или 'ужин с девушкой')",
        'processing': "🔍 Ищу лучшие варианты по всей Москве...",
        'planner_start': "✅ Нашел отличный вариант: **{venue_name}**. Составляю план...",
        'final_greeting': "Напиши /start, чтобы составить новый план!",
        'critical_error': "⚠️ Произошла ошибка. Напиши /start заново.",
        'ask_custom_budget': "Напиши бюджет цифрами (например: 0 или 5000):",
        'ask_custom_people': "Сколько человек?",
        'ask_custom_duration': "На сколько времени?"
    }
}

QUERY_GENERATOR_PROMPT = """
Ты — поисковик мест в Москве. Преврати запрос пользователя в 2-3 ключевых слова для поиска.
Если запрос про театр, кино, музей, стадион — обязательно используй эти слова.
Пример: 'хочу посмотреть картины' -> 'Художественный музей'
Пример: 'покачаться' -> 'Фитнес клуб'
Ответ только фразой.
"""

CLASSIFIER_PROMPT = """
Классифицируй запрос в одну категорию:
- GASTRONOMY (еда, кафе, рестораны)
- SPORT (спорт, залы, стадионы, бег)
- CULTURE (театры, музеи, выставки, кино)
- RELAX (парки, прогулки, спа)
- GENERAL (другое)
Ответ только словом.
"""

CHOOSER_PROMPT = "Выбери ОДНО лучшее место из списка, которое максимально подходит под описание интересов. Ответ строго в формате: Название - Ссылка."

PLANNER_PROMPT = """
Ты — профессиональный гид по Москве EventMe AI. Твоя задача — составить план.

!!! ВАЖНО !!!
Никогда не пиши "я не обучена" или "не могу". Ты всегда находишь решение.

!!! БЮДЖЕТ !!!
1. Если бюджет "БЕСПЛАТНО (0 руб)" — предлагай ТОЛЬКО бесплатные активности (прогулки, бесплатные выставки, просмотр архитектуры). Еда — "взять с собой".
2. Если бюджет есть — предлагай услуги в рамках суммы.

СТРУКТУРА ОТВЕТА (используй Markdown):
1. 🎨 **Концепция**: (Кратко и сочно о сути плана)
2. 📍 **Место**: [Название](Ссылка)
3. 📝 **План действий**: (Тайминг по часам: что делать, куда смотреть)
4. 💸 **Бюджет**: (Расчет на человека)
5. ✨ **Совет**: (Лайфхак для этого места)

Будь краток, позитивен и полезен.
"""

TELEGRAM_BOT_TOKEN = userdata.get('TELEGRAM_BOT_TOKEN')
GIGACHAT_CREDENTIALS = userdata.get('GIGACHAT_CREDENTIALS')

SELECT_LANG, BUDGET, PEOPLE, DURATION, INTERESTS = range(5)

# --- 4. КНОПКИ ---

def get_keyboard(step: int, lang: str) -> InlineKeyboardMarkup:
    if step == BUDGET:
        return InlineKeyboardMarkup([
            [InlineKeyboardButton("Бесплатно (0 ₽)", callback_data="budget_0-0"), InlineKeyboardButton("до 1500 ₽", callback_data="budget_0-1500")],
            [InlineKeyboardButton("1500-5000 ₽", callback_data="budget_1500-5000"), InlineKeyboardButton("Свой вариант", callback_data="budget_custom")]
        ])
    elif step == PEOPLE:
        return InlineKeyboardMarkup([
            [InlineKeyboardButton("Я один", callback_data="people_1"), InlineKeyboardButton("Вдвоем", callback_data="people_2")],
            [InlineKeyboardButton("Компания (3+)", callback_data="people_3+"), InlineKeyboardButton("Свой вариант", callback_data="people_custom")]
        ])
    elif step == DURATION:
        return InlineKeyboardMarkup([
            [InlineKeyboardButton("2-3 часа", callback_data="duration_2-3h"), InlineKeyboardButton("Весь день", callback_data="duration_day")],
            [InlineKeyboardButton("Свой вариант", callback_data="duration_custom")]
        ])
    return None

# --- 5. ЛОГИКА ---

async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    context.user_data.clear()
    await update.message.reply_text(STRINGS['ru']['select_lang'], reply_markup=InlineKeyboardMarkup([[InlineKeyboardButton("Русский 🇷🇺", callback_data='lang_ru')]]))
    return SELECT_LANG

async def button_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    query = update.callback_query
    await query.answer()

    step_str, value = query.data.split('_', 1)

    if step_str == 'lang':
        context.user_data['lang'] = value
        await query.edit_message_text(STRINGS[value]['select_budget'], reply_markup=get_keyboard(BUDGET, value))
        return BUDGET

    if value == 'custom':
        context.user_data['awaiting_input'] = step_str
        ask_text = {'budget': STRINGS['ru']['ask_custom_budget'], 'people': STRINGS['ru']['ask_custom_people'], 'duration': STRINGS['ru']['ask_custom_duration']}
        await query.edit_message_text(ask_text[step_str])
        return {'budget': BUDGET, 'people': PEOPLE, 'duration': DURATION}[step_str]

    context.user_data[step_str] = value
    current_step = {'budget': BUDGET, 'people': PEOPLE, 'duration': DURATION}[step_str]

    if current_step == BUDGET:
        await query.edit_message_text(STRINGS['ru']['select_people'], reply_markup=get_keyboard(PEOPLE, 'ru'))
        return PEOPLE
    elif current_step == PEOPLE:
        await query.edit_message_text(STRINGS['ru']['select_duration'], reply_markup=get_keyboard(DURATION, 'ru'))
        return DURATION
    else:
        await query.edit_message_text(STRINGS['ru']['select_interests'])
        return INTERESTS

async def text_handler(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    step_to_fill = context.user_data.get('awaiting_input')
    if not step_to_fill: return await interests_step(update, context)

    context.user_data[step_to_fill] = update.message.text
    del context.user_data['awaiting_input']

    if step_to_fill == 'budget':
        await update.message.reply_text(STRINGS['ru']['select_people'], reply_markup=get_keyboard(PEOPLE, 'ru'))
        return PEOPLE
    elif step_to_fill == 'people':
        await update.message.reply_text(STRINGS['ru']['select_duration'], reply_markup=get_keyboard(DURATION, 'ru'))
        return DURATION
    else:
        await update.message.reply_text(STRINGS['ru']['select_interests'])
        return INTERESTS

async def interests_step(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_data = context.user_data
    user_data['interests'] = update.message.text

    await update.message.reply_text(STRINGS['ru']['processing'])

    # Расшифровка бюджета
    raw_budget = user_data.get('budget', '0-0')
    human_budget = "Неизвестно"
    if raw_budget == '0-0': human_budget = "СТРОГО БЕСПЛАТНО (0 руб)"
    elif raw_budget == '0-1500': human_budget = "Эконом (до 1500 руб)"
    elif raw_budget == '1500-5000': human_budget = "Средний (до 5000 руб)"
    else: human_budget = raw_budget

    summary = f"Бюджет: {human_budget}, Люди: {user_data.get('people')}, Время: {user_data.get('duration')}, Запрос: {user_data['interests']}"
    print(f"DEBUG: {summary}")

    try:
        async with GigaChat(credentials=GIGACHAT_CREDENTIALS, verify_ssl_certs=False) as giga:
            # 1. Поиск мест (Парсинг + Расширенная база)
            # Генерируем поисковый запрос
            res_query = await giga.achat(payload={"model": "GigaChat:latest", "messages": [{"role": "system", "content": QUERY_GENERATOR_PROMPT}, {"role": "user", "content": user_data['interests']}]})
            search_q = res_query.choices[0].message.content

            venues = perform_yandex_search(search_q)

            # 2. Выбор лучшего места
            chooser_text = "\n".join([f"- {v['title']} ({v['link']}) - {v.get('description','')}" for v in venues])

            res_choose = await giga.achat(payload={"model": "GigaChat-Pro", "messages": [{"role": "system", "content": CHOOSER_PROMPT}, {"role": "user", "content": f"{summary}\nВарианты (выбери один):\n{chooser_text}"}]})
            chosen_text = res_choose.choices[0].message.content

            # Находим выбранное место в списке
            chosen_venue = venues[0]
            for v in venues:
                if v['title'] in chosen_text:
                    chosen_venue = v
                    break

            await update.message.reply_text(STRINGS['ru']['planner_start'].format(venue_name=chosen_venue['title']), parse_mode=ParseMode.MARKDOWN)

            # 3. Генерация плана
            planner_input = f"ДАННЫЕ ПОЛЬЗОВАТЕЛЯ:\n{summary}\n\nВЫБРАННОЕ МЕСТО:\nНазвание: {chosen_venue['title']}\nАдрес/Описание: {chosen_venue.get('description', 'Москва')}\nСсылка: {chosen_venue['link']}"

            res_plan = await giga.achat(payload={"model": "GigaChat-Pro", "messages": [{"role": "system", "content": PLANNER_PROMPT}, {"role": "user", "content": planner_input}], "temperature": 0.7})

            await update.message.reply_text(res_plan.choices[0].message.content, parse_mode=ParseMode.MARKDOWN, disable_web_page_preview=True)
            await update.message.reply_text(STRINGS['ru']['final_greeting'])

    except Exception as e:
        print(f"CRITICAL ERROR: {e}")
        # Если совсем всё упало, пытаемся хоть что-то ответить
        await update.message.reply_text(STRINGS['ru']['critical_error'])

    return ConversationHandler.END

def main():
    app = Application.builder().token(TELEGRAM_BOT_TOKEN).build()

    conv = ConversationHandler(
        entry_points=[CommandHandler('start', start_command)],
        states={
            SELECT_LANG: [CallbackQueryHandler(button_handler, pattern='^lang_')],
            BUDGET: [CallbackQueryHandler(button_handler, pattern='^budget_'), MessageHandler(filters.TEXT, text_handler)],
            PEOPLE: [CallbackQueryHandler(button_handler, pattern='^people_'), MessageHandler(filters.TEXT, text_handler)],
            DURATION: [CallbackQueryHandler(button_handler, pattern='^duration_'), MessageHandler(filters.TEXT, text_handler)],
            INTERESTS: [MessageHandler(filters.TEXT, interests_step)]
        },
        fallbacks=[CommandHandler('cancel', start_command)]
    )

    app.add_handler(conv)
    print("Бот запущен v29.0 (MEGA SEARCH EDITION)")

    loop = asyncio.get_running_loop()
    loop.create_task(app.run_polling())

if TELEGRAM_BOT_TOKEN and GIGACHAT_CREDENTIALS:
    main()
else:
    print("❌ Вставь токены в Secrets!")


/tmp/ipython-input-2982862889.py:332: PTBUserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message. Read this FAQ entry to learn more about the per_* settings: https://github.com/python-telegram-bot/python-telegram-bot/wiki/Frequently-Asked-Questions#what-do-the-per_-settings-in-conversationhandler-do.
  conv = ConversationHandler(


Бот запущен v29.0 (MEGA SEARCH EDITION)
DEBUG: Бюджет: Средний (до 5000 руб), Люди: 3+, Время: day, Запрос: c компанией хотим пойти позаниматься спортом на выходных, потом в центре покушать, а дальше хочется посмотреть зрелища
🚀 Поиск на Yandex.Maps: ''Спорт зал', 'Ресторан центр Москвы', 'Театр Москва''...
🔧 Включаю режим 'Виртуальный гид' (Расширенная база)...
